In [ ]:
import matplotlib.cm as cm
import numpy as np
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

from tqdm import tqdm
from epicare.envs import EpiCare
from epicare.policies import StandardOfCare, ClinicalTrial, Oracle, Random
from epicare.evaluations import run_episode
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FixedLocator

from tueplots import bundles

plt.rcParams.update(bundles.neurips2024())
plt.rcParams["figure.dpi"] = 300

results_directory = "../algorithms/results/"

## Calculate Results for Baseline Policies

In [ ]:
num_seeds = 8
num_replicates = 4
num_episodes = 1000
results_directory = "../algorithms/results/"

all_data = []

with tqdm(total=num_seeds * num_replicates * 4) as pbar:
    for seed in range(1, num_seeds + 1):
        environment = EpiCare(seed=seed)
    
        # Setting up the policies
        policies = {
            "Random": Random(environment),
            "SMART": ClinicalTrial(environment),
            "SoC": StandardOfCare(environment, alpha=0.8),
            "OP": Oracle(environment),
        }
    
        # This dictionary will store the stats for each policy
        policy_stats = {}
    
        # Simulation loop
        for name, policy in policies.items():
            for replicate in range(num_replicates):
                for episode in range(num_episodes):
                    total_reward, time_to_remission, steps, transitions = (
                        run_episode(
                            environment, policy, name, policy_stats, verbose=False
                        )
                    )
                    policy_stats[name]["adverse_event"][-1] /= steps
            
                # After simulation, calculate averages and remission rates
                average_reward = np.mean(policy_stats[name]["total_rewards"]) * 100/64
                reward_std = np.std(policy_stats[name]["total_rewards"]) * 100/64
                remission_rate = np.mean(policy_stats[name]["remission"])
                average_time_to_remission = (
                    np.mean(policy_stats[name]["times_to_remission"])
                    if policy_stats[name]["times_to_remission"]
                    else None
                )
                time_to_remission_std = (
                    np.std(policy_stats[name]["times_to_remission"])
                    if policy_stats[name]["times_to_remission"]
                    else None
                )
                adverse_event_rate = np.mean(policy_stats[name]["adverse_event"])
                adverse_event_rate_std = np.std(policy_stats[name]["adverse_event"])
                # get sem
                adverse_event_rate_sem = adverse_event_rate_std / np.sqrt(num_episodes)
        
                # Adding data to all_data
                all_data.append(
                    {
                        "policy": name,
                        "env_seed": seed,
                        "mean_return": average_reward,
                        "std_return": reward_std,
                        "mean_time_to_remission": average_time_to_remission,
                        "std_time_to_remission": time_to_remission_std,
                        "mean_remission_rate": remission_rate,
                        "mean_adverse_event_rate": adverse_event_rate,
                        "sem_adverse_event_rate": adverse_event_rate_sem,
                    }
                )
                pbar.update()

# Creating the final dataframe
df = pd.DataFrame(all_data)
# Saving separate CSV files for each policy
for policy in df["policy"].unique():
    # make policy name lowercase
    policy_df = df[df["policy"] == policy]
    policy_df = policy_df.drop(columns=["policy"])
    policy_df.to_csv(os.path.join(results_directory, f"{policy.lower()}_results.csv"), index=False)

## Generate Tables

### Table 1: Online Evaluation Results - Mean Episeode Rewards

In [ ]:
models = ["op", "awac", "edac", "td3_bc", "cql", "iql", "dqn", "soc", "bc", "random"]
baseline = [True, False, False, False, False, False, False, True, False, True]
model_names = [
    "OP",
    "AWAC",
    "EDAC",
    "TD3+BC",
    "CQL",
    "IQL",
    "DQN",
    "SoC",
    "BC",
    "Random",
]
model_mapping = dict(zip(models, model_names))
results = []
results_directory = "../algorithms/results/"

# Create a dictionary to store the results for each seed and model
seed_results = {}

for i, model in enumerate(models):
    df = pd.read_csv(os.path.join(results_directory, f"{model}_results.csv"))
    if not baseline[i]:
        df = df[(df['episodes_avail'] == 2**17) & (df['behavior_policy'] == 'smart')]

    # Group results by env_seed
    grouped = df.groupby("env_seed")

    # Calculate mean and standard deviation of mean_return for each seed
    for seed, group in grouped:
        mean = group["mean_return"].mean()
        std = group["mean_return"].std()

        # Store the results in the dictionary
        if seed not in seed_results:
            seed_results[seed] = {}
        seed_results[seed][model] = (mean, std)

    # Calculate overall mean and standard deviation across seeds for the model
    overall_mean = df["mean_return"].mean()
    overall_std = df.groupby("env_seed")["mean_return"].std().mean()

    results.append((model, overall_mean, overall_std))

# Sort all models based on overall mean performance
results.sort(key=lambda x: x[1])

# Reorder to place baseline models first
sorted_results = [
    res for res in results if model_mapping[res[0]] in ["OP", "SoC", "Random"]
] + [res for res in results if model_mapping[res[0]] not in ["OP", "SoC", "Random"]]

# Create a DataFrame for the results
data = {}

# Add overall mean and standard deviation for each model
mean_row = []
for _, mean, std in sorted_results:
    if np.isnan(std):
        mean_row.append("{:.1f}".format(mean))
    else:
        mean_row.append("{:.1f} ({:.1f})".format(mean, std))
data["Mean"] = mean_row

# Add mean and standard deviation for each seed
for seed in sorted(seed_results.keys()):
    seed_row = []
    for model, _, _ in sorted_results:
        mean, std = seed_results[seed].get(
            model, (0, np.nan)
        )  # Default to (0, NaN) if not available
        if np.isnan(std):
            seed_row.append("{:.1f}".format(mean))
        else:
            seed_row.append("{:.1f} ({:.1f})".format(mean, std))
    data[f"Seed {seed}"] = seed_row

# Convert dictionary to DataFrame
df_results = pd.DataFrame(
    data, index=[model_mapping[model] for model, _, _ in sorted_results]
)

print(df_results)

### Table 2: Online Evaluation Results - Adverse Event Rates

In [ ]:
models = ["op", "awac", "edac", "td3_bc", "cql", "iql", "dqn", "soc", "bc", "random"]
baseline = [True, False, False, False, False, False, False, True, False, True]
model_names = [
    "OP",
    "AWAC",
    "EDAC",
    "TD3+BC",
    "CQL",
    "IQL",
    "DQN",
    "SoC",
    "BC",
    "Random",
]
model_mapping = dict(zip(models, model_names))
results = []
results_directory = "../algorithms/results/"

# Create a dictionary to store the results for each seed and model
seed_results = {}

for i, model in enumerate(models):
    df = pd.read_csv(os.path.join(results_directory, f"{model}_results.csv"))
    if not baseline[i]:
        df = df[(df["episodes_avail"] == 2**17) & (df["behavior_policy"] == "smart")]

    # Group results by env_seed
    grouped = df.groupby("env_seed")
    # Calculate mean and standard deviation of mean_adverse_event_rate for each seed
    for seed, group in grouped:
        mean = (
            group["mean_adverse_event_rate"].mean() * 10000
        )  # Convert percentage to per 10,000
        std = (
            group["mean_adverse_event_rate"].std() * 10000
        )  # Convert percentage to per 10,000
        # Store the results in the dictionary
        if seed not in seed_results:
            seed_results[seed] = {}
        seed_results[seed][model] = (mean, std)
    # Calculate overall mean and standard deviation across seeds for the model
    overall_mean = (
        df["mean_adverse_event_rate"].mean() * 10000
    )  # Convert percentage to per 10,000
    overall_std = (
        df.groupby("env_seed")["mean_adverse_event_rate"].std().mean() * 10000
    )  # Convert percentage to per 10,000
    results.append((model, overall_mean, overall_std))

# Sort all models based on overall mean performance
results.sort(key=lambda x: -x[1])
# Reorder to place baseline models first
sorted_results = [
    res for res in results if model_mapping[res[0]] in ["OP", "SoC", "Random"]
] + [res for res in results if model_mapping[res[0]] not in ["OP", "SoC", "Random"]]

# Create a DataFrame for the results
data = {}

# Add overall mean and standard deviation for each model
mean_row = []
for _, mean, std in sorted_results:
    if np.isnan(std):
        mean_row.append("{:.0f}".format(mean))
    else:
        mean_row.append("{:.0f} ({:.0f})".format(mean, std))
data["Mean"] = mean_row

# Add mean and standard deviation for each seed
for seed in sorted(seed_results.keys()):
    seed_row = []
    for model, _, _ in sorted_results:
        mean, std = seed_results[seed].get(
            model, (0, np.nan)
        )  # Default to (0, NaN) if not available
        if np.isnan(std):
            seed_row.append("{:.0f}".format(mean))
        else:
            seed_row.append("{:.0f} ({:.0f})".format(mean, std))
    data[f"Seed {seed}"] = seed_row

# Convert dictionary to DataFrame
df_results = pd.DataFrame(
    data, index=[model_mapping[model] for model, _, _ in sorted_results]
)

print(df_results)

### Table 6: Online Evalutaion Results - Mean Remission Rate

In [ ]:
models = ["op", "awac", "edac", "td3_bc", "cql", "iql", "dqn", "soc", "bc", "random"]
baseline = [True, False, False, False, False, False, False, True, False, True]
model_names = [
    "OP",
    "AWAC",
    "EDAC",
    "TD3+BC",
    "CQL",
    "IQL",
    "DQN",
    "SoC",
    "BC",
    "Random",
]
model_mapping = dict(zip(models, model_names))
results = []
results_directory = "../algorithms/results/"

# Create a dictionary to store the results for each seed and model
seed_results = {}

for i, model in enumerate(models):
    df = pd.read_csv(os.path.join(results_directory, f"{model}_results.csv"))
    if not baseline[i]:
        df = df[(df["episodes_avail"] == 2**17) & (df["behavior_policy"] == "smart")]

    # Group results by env_seed
    grouped = df.groupby("env_seed")

    # Calculate mean and standard deviation of mean_remission_rate for each seed
    for seed, group in grouped:
        mean = group["mean_remission_rate"].mean()
        std = group["mean_remission_rate"].std()

        # Store the results in the dictionary
        if seed not in seed_results:
            seed_results[seed] = {}
        seed_results[seed][model] = (mean, std)

    # Calculate overall mean and standard deviation across seeds for the model
    overall_mean = df["mean_remission_rate"].mean()
    overall_std = df.groupby("env_seed")["mean_remission_rate"].std().mean()

    results.append((model, overall_mean, overall_std))

# Sort all models based on overall mean performance
results.sort(key=lambda x: x[1])

# Reorder to place baseline models first
sorted_results = [
    res for res in results if model_mapping[res[0]] in ["OP", "SoC", "Random"]
] + [res for res in results if model_mapping[res[0]] not in ["OP", "SoC", "Random"]]

# Create a DataFrame for the results
data = {}

# Add overall mean and standard deviation for each model
mean_row = []
for _, mean, std in sorted_results:
    if np.isnan(std):
        mean_row.append("{:.2f}".format(mean))
    else:
        mean_row.append("{:.2f}({:.2f})".format(mean, std))
data["Mean"] = mean_row

# Add mean and standard deviation for each seed
for seed in sorted(seed_results.keys()):
    seed_row = []
    for model, _, _ in sorted_results:
        mean, std = seed_results[seed].get(
            model, (0, np.nan)
        )  # Default to (0, NaN) if not available
        if np.isnan(std):
            seed_row.append("{:.2f}".format(mean))
        else:
            seed_row.append("{:.2f}({:.2f})".format(mean, std))
    data[f"Seed {seed}"] = seed_row

# Convert dictionary to DataFrame
df_results = pd.DataFrame(
    data, index=[model_mapping[model] for model, _, _ in sorted_results]
)

print(df_results)

### Table 7: Online Evaluation Results - Time to Remission

In [ ]:
models = ["op", "awac", "edac", "td3_bc", "cql", "iql", "dqn", "soc", "bc", "random"]
baseline = [True, False, False, False, False, False, False, True, False, True]
model_names = [
    "OP",
    "AWAC",
    "EDAC",
    "TD3+BC",
    "CQL",
    "IQL",
    "DQN",
    "SoC",
    "BC",
    "Random",
]
model_mapping = dict(zip(models, model_names))
results = []
results_directory = "../algorithms/results/"

# Create a dictionary to store the results for each seed and model
seed_results = {}

for i, model in enumerate(models):
    df = pd.read_csv(os.path.join(results_directory, f"{model}_results.csv"))
    if not baseline[i]:
        df = df[(df["episodes_avail"] == 2**17) & (df["behavior_policy"] == "smart")]

    # Group results by env_seed
    grouped = df.groupby("env_seed")

    # Calculate mean and standard deviation of mean_time_to_remission for each seed
    for seed, group in grouped:
        mean = group["mean_time_to_remission"].mean()
        std = group["mean_time_to_remission"].std()

        # Store the results in the dictionary
        if seed not in seed_results:
            seed_results[seed] = {}
        seed_results[seed][model] = (mean, std)

    # Calculate overall mean and standard deviation across seeds for the model
    overall_mean = df["mean_time_to_remission"].mean()
    overall_std = df.groupby("env_seed")["mean_time_to_remission"].std().mean()

    results.append((model, overall_mean, overall_std))

# Sort all models based on overall mean performance
results.sort(key=lambda x: -x[1])

# Reorder to place baseline models first
sorted_results = [
    res for res in results if model_mapping[res[0]] in ["OP", "SoC", "Random"]
] + [res for res in results if model_mapping[res[0]] not in ["OP", "SoC", "Random"]]

# Create a DataFrame for the results
data = {}

# Add overall mean and standard deviation for each model
mean_row = []
for _, mean, std in sorted_results:
    if np.isnan(std):
        mean_row.append("{:.1f}".format(mean))
    else:
        mean_row.append("{:.1f}({:.1f})".format(mean, std))
data["Mean"] = mean_row

# Add mean and standard deviation for each seed
for seed in sorted(seed_results.keys()):
    seed_row = []
    for model, _, _ in sorted_results:
        mean, std = seed_results[seed].get(
            model, (0, np.nan)
        )  # Default to (0, NaN) if not available
        if np.isnan(std):
            seed_row.append("{:.1f}".format(mean))
        else:
            seed_row.append("{:.1f}({:.1f})".format(mean, std))
    data[f"Seed {seed}"] = seed_row

# Convert dictionary to DataFrame
df_results = pd.DataFrame(
    data, index=[model_mapping[model] for model, _, _ in sorted_results]
)

print(df_results)

### Table 8: Online Evaluation Results - Behavior Policy Comparison (SoC vs. SMART)

In [ ]:
models = ["edac", "awac", "bc", "td3_bc", "iql", "dqn", "cql"]
model_names = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]
model_mapping = dict(zip(models, model_names))
results_directory = "../algorithms/results/"

# Create a dictionary to store the results for each policy type and model
policy_results = {"smart": {}, "soc": {}}

for model in models:
    for policy in ["smart", "soc"]:
        df = pd.read_csv(os.path.join(results_directory, f"{model}_results.csv"))
        if policy == "smart":
            df = df[
                (df["episodes_avail"] == 2**17) & (df["behavior_policy"] == "smart")
            ]
        else:
            df = df[df["behavior_policy"] == "soc"]

        # Filter for env_seed == 1
        df = df[df["env_seed"] == 1]

        # Calculate mean and standard deviation of mean_time_to_remission
        mean = df["mean_return"].mean()
        std = df["mean_return"].std()

        # Store the results in the dictionary
        if model not in policy_results[policy]:
            policy_results[policy][model] = (mean, std)

# Create a DataFrame for the results
data = {"Model": model_names}
smart_means = []
smart_stds = []
soc_means = []
soc_stds = []

for model in models:
    mean, std = policy_results["smart"].get(model, (0, np.nan))
    smart_means.append(f"{mean:.1f}")
    smart_stds.append(f"{std:.1f}")
    mean, std = policy_results["soc"].get(model, (0, np.nan))
    soc_means.append(f"{mean:.1f}")
    soc_stds.append(f"{std:.1f}")

data["SMART Mean (Std)"] = [
    f"{mean}({std})" for mean, std in zip(smart_means, smart_stds)
]
data["SoC Mean (Std)"] = [f"{mean}({std})" for mean, std in zip(soc_means, soc_stds)]

df_results = pd.DataFrame(data)
df_results.set_index("Model", inplace=True)

print(df_results)

### Figure 3: Data Restriction Trials - Median Episode Reward

In [ ]:
models = ["cql", "iql", "dqn", "td3_bc"]  # List of models to plot
labels = ["CQL", "IQL", "DQN", "TD3+BC"]  # List of labels for the legend

# Set the Seaborn color palette to the colorblind palette
sns.set_palette("Set2")

# Adding a vertical line at 2876 patients
plt.axvline(x=2876, color="C4", linestyle="dotted")
plt.text(2876+200, 13-11, 'Largest Ever SMART', color='C4', rotation=90, verticalalignment='top', fontsize=7)

# Adding horizontal lines for typical SoC performance
plt.axhline(y=95.7, color="k", linestyle="--", label="OP")
plt.axhline(y=44.2, color="C5", linestyle="--", label="SoC")
plt.axhline(y=5.5, color="gray", linestyle="--", label="Random")

for model in models:
    df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
    # Restrict dataframe to only seed 1 and 'smart' behavior policy
    df = df[(df['env_seed'] == 1) & (df['behavior_policy'] == 'smart') ]
    grouped = df.groupby(["episodes_avail"])
    median = grouped["mean_return"].median()
    min = grouped["mean_return"].min()
    max = grouped["mean_return"].max()
    std = grouped["mean_return"].std()

    # Plotting mean
    plt.plot(
        median.index, median, marker="o", label=labels[models.index(model)], markersize=4
    )

    # Adding shaded error (std deviation)
    plt.fill_between(median.index, min, max, alpha=0.4)

# Setting a logarithmic scale for the x-axis
plt.xscale("log")

# Labels and title
plt.xlabel("Episodes Available")
plt.ylabel("Median Episode Reward")

# Making the legend readable and not overlap with the plot
plt.legend(title="Model/Policy", loc="lower right")

# Fine-tuning the appearance
plt.tight_layout()  # Adjusts subplot params so that the subplot(s) fits into the figure area

# Saving the figure
plt.savefig("data_restriction_reward.pdf")

# Display the plot
plt.show()

### Figure 7: Baseline Policy Comparison

In [ ]:
num_seeds = 100  # Number of random seeds
num_episodes = 1000  # Number of evaluation episodes

# Initialize containers for aggregated data
all_rewards = []
all_remission_rates = []
all_times_to_remission = []
all_adverse_rates = []
num_steps = 0
num_transitions = 0

with tqdm(total=3 * num_episodes * num_seeds) as pbar:
    for seed in range(num_seeds):
        environment = EpiCare(seed=seed)

        # Setting up the policies
        policy_stats = {}
        policies = {
            "Random": Random(environment),
            "SMART": ClinicalTrial(environment),
            "SoC": StandardOfCare(environment),
        }

        # Simulation loop
        policy_stats = {}
        for name, policy in policies.items():
            for episode in range(num_episodes):
                total_reward, time_to_remission, steps, transitions = run_episode(
                    environment, policy, name, policy_stats, verbose=False
                )
                policy_stats[name]["adverse_event"][-1] /= steps
                num_steps += steps
                num_transitions += transitions
                pbar.update()

        # After simulation, calculate averages and remission rates
        for stats in policy_stats.values():
            stats["average_reward"] = np.mean(stats["total_rewards"])
            stats["reward_std"] = np.std(stats["total_rewards"])
            stats["remission_rate"] = np.mean(stats["remission"])
            stats["adverse_rate"] = np.mean(stats["adverse_event"])
            if stats["times_to_remission"]:
                stats["average_time_to_remission"] = np.mean(stats["times_to_remission"])
            else:
                stats["average_time_to_remission"] = None

        policy_names = list(policy_stats.keys())

        # Process and store data for each seed
        df_rewards = pd.DataFrame.from_records(
            [policy_stats[name]["total_rewards"] for name in policy_names]
        ).T
        df_rewards.columns = policy_names
        all_rewards.append(df_rewards)

        remission_rates = [
            np.average(policy_stats[name]["remission_rate"]) for name in policy_names
        ]
        all_remission_rates.append(remission_rates)
        
        adverse_rates = [
            np.average(policy_stats[name]["adverse_rate"]) for name in policy_names
        ]
        all_adverse_rates.append(adverse_rates)

        df_times_to_remission = pd.DataFrame.from_records(
            [policy_stats[name]["times_to_remission"] for name in policy_names]
        ).T
        df_times_to_remission.columns = policy_names
        df_times_to_remission = df_times_to_remission.astype("Int64")
        all_times_to_remission.append(df_times_to_remission)

# Aggregate data across all seeds
agg_rewards = (pd.concat(all_rewards) / 64) * 100
agg_remission_rates = pd.DataFrame(all_remission_rates, columns=policy_names)
agg_times_to_remission = pd.concat(all_times_to_remission)
agg_adverse_rates = pd.DataFrame(all_adverse_rates, columns=policy_names)

# Boxplot setup
fig, axes = plt.subplots(1, 4, figsize=(6.5, 2.0), width_ratios=[2,1,2,1])

# Aggregate plots
# Plot 1: Total Reward
sns.violinplot(data=agg_rewards, ax=axes[0], bw_adjust=0.1, inner=None)
# Mark the mean for each policy
for i, name in enumerate(policy_names):
    axes[0].plot(i, agg_rewards[name].mean(), "o", color="black", markersize=4)
axes[0].set_ylabel("Total Episode Reward")

# Plot 2: Remission Rate
sns.barplot(data=agg_remission_rates, ax=axes[1])
axes[1].set_ylabel("Remission Rate")

# Plot 3: Time to Remission
sns.histplot(
    data=agg_times_to_remission, ax=axes[2], multiple="dodge", discrete=True, shrink=0.8
)
axes[2].set_ylabel("Count")
axes[2].yaxis
axes[2].set_xlabel("Time to Remission")
axes[2].set_xticks(range(1, environment.max_visits + 1))
axes[2].axes.yaxis.set_major_formatter(lambda value, text: str(int(value/1000))+"k" if value > 0 else "0")

# Plot 4: Adverse Event Rate
sns.barplot(data=agg_adverse_rates, ax=axes[3])
axes[3].set_ylabel("Adverse Event Rate")

# Fine-tuning the aesthetics for publication quality
for ax in axes:
    if ax != axes[2]:
        ax.tick_params(axis="x", labelrotation=45)
    ax.tick_params(axis="both", which="major")
    if ax == axes[1]:
        ax.set_ylim(bottom=0)

# Adjusting the layout to prevent overlap
plt.tight_layout()

plt.savefig("compare_policies.pdf")

# Display plots
plt.show()

print(
    "Number of transitions per episode: ",
    num_transitions / (num_episodes * num_seeds * 3),
)

### Figure 8: Tuning SoC Policy - alpha

In [ ]:
num_seeds = 100  # Number of random seeds
num_episodes = 1000  # Number of separate evaluation episodes

# Initialize containers for aggregated data
all_rewards = []
all_remission_rates = []
all_times_to_remission = []
all_adverse_rates = []
num_steps = 0
num_transitions = 0

with tqdm(total=11*num_episodes*num_seeds) as pbar:
    for seed in range(num_seeds):
        environment = EpiCare(seed=seed)

        # Setting up the policies
        policy_stats = {}
        # SoC with all alphas between 0 and 1, in increments of 0.1
        policies = {
            f"{alpha:.1f}": StandardOfCare(environment, alpha=alpha)
            for alpha in [i * 0.1 for i in range(11)]
        }

        # Simulation loop
        policy_stats = {}
        for name, policy in policies.items():
            for episode in range(num_episodes):
                total_reward, time_to_remission, steps, transitions = run_episode(
                    environment, policy, name, policy_stats, verbose=False
                )
                num_steps += steps
                num_transitions += transitions
                pbar.update()

        # After simulation, calculate averages and remission rates
        for stats in policy_stats.values():
            stats["average_reward"] = np.mean(stats["total_rewards"])
            stats["reward_std"] = np.std(stats["total_rewards"])
            stats["remission_rate"] = np.mean(stats["remission"])
            stats["adverse_rate"] = np.mean(stats["adverse_event"])
            if stats["times_to_remission"]:
                stats["average_time_to_remission"] = np.mean(stats["times_to_remission"])
            else:
                stats["average_time_to_remission"] = None

        policy_names = list(policy_stats.keys())

        # Process and store data for each seed
        df_rewards = pd.DataFrame.from_records(
            [policy_stats[name]["total_rewards"] for name in policy_names]
        ).T
        df_rewards.columns = policy_names
        all_rewards.append(df_rewards)

        remission_rates = [
            np.average(policy_stats[name]["remission_rate"]) for name in policy_names
        ]
        all_remission_rates.append(remission_rates)
        
        adverse_rates = [
            np.average(policy_stats[name]["adverse_rate"]) for name in policy_names
        ]
        all_adverse_rates.append(adverse_rates)

        df_times_to_remission = pd.DataFrame.from_records(
            [policy_stats[name]["times_to_remission"] for name in policy_names]
        ).T
        df_times_to_remission.columns = policy_names
        df_times_to_remission = df_times_to_remission.astype("Int64")
        all_times_to_remission.append(df_times_to_remission)

# Aggregate data across all seeds
agg_rewards = (pd.concat(all_rewards) / 64) * 100
agg_remission_rates = pd.DataFrame(all_remission_rates, columns=policy_names)
agg_times_to_remission = pd.concat(all_times_to_remission)
agg_adverse_rates = pd.DataFrame(all_adverse_rates, columns=policy_names)

# Generate a list of alpha values as strings for the x-axis labels
fig, axes = plt.subplots(1, 3, figsize=(6.5, 2.0))

# Plot 1: Total Reward
sns.violinplot(
    data=agg_rewards, ax=axes[0], bw_adjust=0.1, cut=0.0, inner=None, linewidth=0.5
)
for i, alpha in enumerate(policies):
    axes[0].plot(i, agg_rewards[alpha].mean(), "o", color="black", markersize=2)
axes[0].set_ylabel("Total Episode Reward")
axes[0].set_xlabel("$\\alpha$")

# Plot 2: Remission Rate
sns.barplot(data=agg_remission_rates, ax=axes[1])
axes[1].set_ylabel("Remission Rate")
axes[1].set_xlabel("$\\alpha$")

# Plot 3: Time to Remission
sns.histplot(
    data=agg_times_to_remission.melt(var_name="Policy", value_name="Time_to_Remission"),
    x="Time_to_Remission",
    hue="Policy",
    ax=axes[2],
    multiple="dodge",
    discrete=True,
    shrink=0.9,
    linewidth=0.2,
)
axes[2].set_ylabel("Count")
axes[2].set_xlabel("Time to Remission")
axes[2].set_xticks(range(1, environment.max_visits + 1))
# remove legend completely
axes[2].get_legend().remove()
axes[2].axes.yaxis.set_major_formatter(lambda value, text: str(int(value/1000))+"k" if value > 0 else "0")

# Fine-tuning the aesthetics for publication quality
for ax in axes:
    ax.tick_params(axis="x", labelrotation=0)
    ax.tick_params(axis="both", which="major")
    if ax == axes[1]:
        ax.set_ylim(bottom=0)

# Adjusting the layout to prevent overlap
plt.tight_layout()

plt.savefig("SoC_alpha.pdf")

# Display plots
plt.show()

### Figure 8: Tuning SoC Policy - kappa

In [ ]:
num_seeds = 100  # Number of random seeds
num_episodes = 10000  # Number of separate evaluation episodes

# Initialize containers for aggregated data
all_rewards = []
all_remission_rates = []
all_times_to_remission = []
all_adverse_rates = []
num_steps = 0
num_transitions = 0

with tqdm(total=6*num_episodes*num_seeds) as pbar:
    for seed in range(num_seeds):
        environment = EpiCare(seed=seed)

        # Setting up the policies
        policy_stats = {}
        # SoC with all kappas between 0 and 0.5, in increments of 0.1
        policies = {
            f"{kappa:.1f}": StandardOfCare(environment, kappa=kappa)
            for kappa in [i * 0.1 for i in range(6)]
        }

        # Simulation loop
        policy_stats = {}
        for name, policy in policies.items():
            for episode in range(num_episodes):
                total_reward, time_to_remission, steps, transitions = run_episode(
                    environment, policy, name, policy_stats, verbose=False
                )
                policy_stats[name]["adverse_event"][-1] /= steps
                num_steps += steps
                num_transitions += transitions
                pbar.update()

        # After simulation, calculate averages and remission rates
        for stats in policy_stats.values():
            stats["average_reward"] = np.mean(stats["total_rewards"])
            stats["reward_std"] = np.std(stats["total_rewards"])
            stats["remission_rate"] = np.mean(stats["remission"])
            stats["adverse_rate"] = np.mean(stats["adverse_event"])
            if stats["times_to_remission"]:
                stats["average_time_to_remission"] = np.mean(stats["times_to_remission"])
            else:
                stats["average_time_to_remission"] = None

        policy_names = list(policy_stats.keys())

        # Process and store data for each seed
        df_rewards = pd.DataFrame.from_records(
            [policy_stats[name]["total_rewards"] for name in policy_names]
        ).T
        df_rewards.columns = policy_names
        all_rewards.append(df_rewards)

        remission_rates = [
            np.average(policy_stats[name]["remission_rate"]) for name in policy_names
        ]
        all_remission_rates.append(remission_rates)
        
        adverse_rates = [
            np.average(policy_stats[name]["adverse_rate"]) for name in policy_names
        ]
        all_adverse_rates.append(adverse_rates)

        df_times_to_remission = pd.DataFrame.from_records(
            [policy_stats[name]["times_to_remission"] for name in policy_names]
        ).T
        df_times_to_remission.columns = policy_names
        df_times_to_remission = df_times_to_remission.astype("Int64")
        all_times_to_remission.append(df_times_to_remission)

# Aggregate data across all seeds, then plot
agg_rewards = (pd.concat(all_rewards) / 64) * 100
agg_remission_rates = pd.DataFrame(all_remission_rates, columns=policy_names)
agg_times_to_remission = pd.concat(all_times_to_remission)
agg_adverse_rates = pd.DataFrame(all_adverse_rates, columns=policy_names)
fig, axes = plt.subplots(1, 4, figsize=(6.5, 2.0))

# Plot 1: Total Reward
sns.violinplot(
    data=agg_rewards, ax=axes[0], bw_adjust=0.1, cut=0.0, inner=None, linewidth=0.5
)
for i, kappa in enumerate(policies):
    axes[0].plot(i, agg_rewards[kappa].mean(), "o", color="black", markersize=2)
axes[0].set_ylabel("Total Episode Reward")
axes[0].set_xlabel("$\\kappa$")

# Plot 2: Remission Rate
sns.barplot(data=agg_remission_rates, ax=axes[1])
axes[1].set_ylabel("Remission Rate")
axes[1].set_xlabel("$\\kappa$")

# Plot 3: Time to Remission
sns.histplot(
    data=agg_times_to_remission.melt(var_name="Policy", value_name="Time_to_Remission"),
    x="Time_to_Remission",
    hue="Policy",
    ax=axes[2],
    multiple="dodge",
    discrete=True,
    shrink=0.9,
    linewidth=0.2,
)
axes[2].set_ylabel("Count")
axes[2].set_xlabel("Time to Remission")
axes[2].set_xticks(range(1, environment.max_visits + 1))
# remove legend completely
axes[2].get_legend().remove()
axes[2].axes.yaxis.set_major_formatter(lambda value, text: str(int(value/1000))+"k" if value > 0 else "0")

# Plot 4: Adverse Event Rate
sns.barplot(data=agg_adverse_rates, ax=axes[3])
axes[3].set_ylabel("Adverse Events per Step")
axes[3].set_xlabel("$\\kappa$")

# Fine-tuning the aesthetics for publication quality
for ax in axes:
    ax.tick_params(axis="x", labelrotation=0)
    ax.tick_params(axis="both", which="major")
    if ax == axes[1]:
        ax.set_ylim(bottom=0)

# Adjusting the layout to prevent overlap
plt.tight_layout()

plt.savefig("SoC_kappa.pdf")

# Display plots
plt.show()

### Figure 11: Data Restriction Trials - Median Adverse Event Rate

In [ ]:
models = ["cql", "iql", "dqn", "td3_bc"]  # List of models to plot
labels = ["CQL", "IQL", "DQN", "TD3+BC"]  # List of labels for the legend

# Set the Seaborn color palette to the colorblind palette
sns.set_palette("Set2")

# Adding a vertical line at 2876 patients
plt.axvline(x=2876, color="C4", linestyle="dotted")
plt.text(2876 - 675, 64, 'Largest Ever SMART', color='C4', rotation=90, verticalalignment='bottom', fontsize=8)

# Adding horizontal lines for typical SoC performance
plt.axhline(y=0, color="k", linestyle="--", label="OP")
plt.axhline(y=15, color="C5", linestyle="--", label="SoC")
plt.axhline(y=56, color="gray", linestyle="--", label="Random")

for model in models:
    df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
    # Restrict dataframe to only seed 1 and 'smart' behavior policy
    df = df[(df["env_seed"] == 1) & (df["behavior_policy"] == "smart")]
    grouped = df.groupby(["episodes_avail"])
    median = grouped["mean_adverse_event_rate"].median()*10000
    min = grouped["mean_adverse_event_rate"].min() * 10000
    max = grouped["mean_adverse_event_rate"].max() * 10000
    std = grouped["mean_adverse_event_rate"].std()

    # Plotting mean
    plt.plot(
        median.index, median, marker="o", label=labels[models.index(model)], markersize=4
    )

    # Adding shaded error (std deviation)
    plt.fill_between(median.index, min, max, alpha=0.4)

# Setting a logarithmic scale for the x-axis
plt.xscale("log")

# Labels and title
plt.xlabel("Episodes Available")
plt.ylabel("Median Adverse Events per 10k Episodes")

# Making the legend readable and not overlap with the plot
plt.legend(title="Model/Policy", loc="upper right")

# Fine-tuning the appearance
plt.tight_layout()  # Adjusts subplot params so that the subplot(s) fits into the figure area

# Saving the figure
plt.savefig("data_restriction_aer.pdf")

# Display the plot
plt.show()

### Figure 14: OPE RMSE vs. Action Choice

In [ ]:
# Define OPE methods and models
models = ["edac", "awac", "bc", "td3_bc", "iql", "dqn", "cql"]
model_labels = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]
ope_methods = ["is", "wis", "pdis", "wpdis"]
colors = ["C" + str(i) for i in range(len(models))]
color_dict = dict(zip(models, colors))
model_labes_dict = dict(zip(models, model_labels))

# Initialize lists to store mean log_probs, their standard deviations, and average RMSE for each model
mean_log_probs = []
std_devs = []
average_rmse_values = []

# Process data for each model to get log_probs and calculate RMSE
for model in models:
    df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
    # Filter rows for checkpoint_32.pt
    if "checkpoint" in df.columns:
        df_filtered = df[df["checkpoint"] == "checkpoint_31.pt"]
        abs_log_probs = df_filtered["mean_meanlogprob_estimate"].abs()
        mean_log_prob = abs_log_probs.mean()
        std_dev = abs_log_probs.std()
    else:
        mean_log_prob = std_dev = np.nan

    mean_log_probs.append(mean_log_prob)
    std_devs.append(std_dev)

    # Calculate RMSE for each model across all OPE methods
    rmse_values = []
    for method in ope_methods:
        online_means = df["mean_return"]
        ope_means = df[f"mean_{method}_estimate"]
        rmse = np.sqrt(
            np.mean(
                (np.clip(ope_means, -100, 100) - np.clip(online_means, -100, 100)) ** 2
            )
        )
        rmse_values.append(rmse)
    average_rmse = np.mean(rmse_values)
    average_rmse_values.append(average_rmse)

# Combine models, their mean log_probs, std_devs, and average RMSE into a DataFrame for sorting
model_data = pd.DataFrame(
    {
        "Model": models,
        "MeanLogProb": mean_log_probs,
        "StdDev": std_devs,
        "AverageRMSE": average_rmse_values,
    }
)

# Sort by AverageRMSE from smallest to largest
model_data_sorted = model_data.sort_values(by="AverageRMSE")

# Create a bar graph for Average RMSE
fig, ax1 = plt.subplots()
ax1.bar(
    model_data_sorted["Model"],
    model_data_sorted["AverageRMSE"],
    color=[color_dict[model] for model in model_data_sorted["Model"]],
    capsize=5,
)
ax1.set_xlabel("Model")
ax1.set_ylabel("Average OPE RMSE", color="k")
ax1.tick_params(axis="y", labelcolor="k")
ax1.set_xticklabels([model_labes_dict[model] for model in model_data_sorted["Model"]])

# Add a secondary y-axis for Mean Abs Log Probability as a line graph
ax2 = ax1.twinx()
ax2.plot(
    model_data_sorted["Model"],
    np.e ** -model_data_sorted["MeanLogProb"],
    color="C" + str(len(model)),
    marker="o",
    linestyle="-",
    linewidth=2,
    markersize=5,
)
ax2.set_ylabel(
    "Average Probability of Training Action",
    color="C" + str(len(model)),
    rotation=-90,
    labelpad=13,
)
ax2.tick_params(axis="y", labelcolor="C" + str(len(model)))

plt.tight_layout()
plt.savefig("avg_probs_and_rmse.pdf")  # Save as high-resolution image
plt.show()

In [ ]:
ope_methods = ["is", "wis", "pdis", "wpdis", "direct"]
ope_labels = ["IS", "WIS", "PDIS", "WPDIS", "DM"]
models = ["edac","awac", "bc", "td3_bc", "iql", "dqn", "cql-dqn"]
model_labels = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]

# Create a figure with subplots
fig, axes = plt.subplots(
    ncols=len(models), nrows=len(ope_methods), figsize=(6.75, 5.5)
)  # Adjust size as needed

# Initialize a dictionary to store RMSE values for each model
model_rmse = {model: [] for model in models}

# Process data for each model and each OPE method
for col, model in enumerate(models):
    for row, method in enumerate(ope_methods):
        df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
        online_means = df["mean_return"]
        ope_means = df[f"mean_{method}_estimate"]

        # Clip the values to be between -100 and 100
        online_means_clipped = np.clip(online_means, -100, 100)
        ope_means_clipped = np.clip(ope_means, -100, 100)

        # Calculate RMSE and store it
        rmse = np.sqrt(np.mean((online_means - ope_means) ** 2))
        model_rmse[model].append(rmse)

        # Plot scatter for each method and model
        axes[row, col].scatter(online_means_clipped, ope_means_clipped, s=5, alpha=0.5)
        axes[row, col].set_aspect("equal", adjustable="box")
        axes[row, col].set_xlim([-75, 100])
        axes[row, col].set_ylim([-75, 100])
        axes[row, col].plot([-75, 100], [-75, 100], color="black")  # y=x line
        axes[row, col].set_xticks([])
        axes[row, col].set_yticks([])

        # Add RMSE as an insert, centered below the scatter plot
        axes[row, col].text(
            0.5,
            1.3,
            f"RMSE: {rmse:0.2f}",
            transform=axes[row, col].transAxes,
            horizontalalignment="center",
            verticalalignment="top",
            bbox=dict(boxstyle="round,pad=0.3", alpha=1.0, facecolor="white"),
        )

for row in range(len(ope_methods)):
    axes[row, 0].set_ylabel(ope_labels[row] + " Estimate")
    axes[row, 0].set_yticks([-75, 0, 100])

for col in range(len(models)):
    axes[0, col].set_title(model_labels[col].upper(), pad=20)
    axes[-1, col].set_xlabel("Online Return")
    axes[-1, col].xaxis.tick_bottom()
    axes[-1, col].set_xticks([-75, 0, 100])

plt.tight_layout()
plt.savefig("ope_scatter.pdf")  # Save as high-resolution image
plt.show()

# Calculate and print average RMSE for each model
average_rmse = {model: np.mean(rmses) for model, rmses in model_rmse.items()}
print("Average RMSE for each model:", average_rmse)

In [ ]:
ope_methods = ["is", "wis", "pdis", "wpdis", "direct"]
ope_labels = ["IS", "WIS", "PDIS", "WPDIS", "DM"]
models = ["edac", "awac", "bc", "td3_bc", "iql", "dqn", "cql-dqn"]
model_labels = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]

# Create a figure with subplots
fig, axes = plt.subplots(
    ncols=len(models), nrows=len(ope_methods), figsize=(6.75, 5.5)
)  # Adjust size as needed

# Initialize a dictionary to store RMSE values for each model
model_rmse = {model: [] for model in models}

# Process data for each model and each OPE method
for col, model in enumerate(models):
    for row, method in enumerate(ope_methods):
        df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
        online_error = np.abs(df["mean_return"] - df[f"mean_{method}_estimate"])
        ope_stds = df[f"std_{method}_estimate"]

        # Plot scatter for each method and model
        axes[row, col].scatter(online_error, ope_stds, s=5, alpha=0.5)
        axes[row, col].set_xlim([0, 100])
        axes[row, col].set_ylim([0, 100])
        axes[row, col].set_xticks([])
        axes[row, col].set_yticks([])

for row in range(len(ope_methods)):
    axes[row, 0].set_ylabel("Std " + ope_labels[row] + " Estimate")
    axes[row, 0].set_yticks([0, 200])

for col in range(len(models)):
    axes[0, col].set_title(model_labels[col].upper(), pad=20)
    axes[-1, col].set_xlabel("OPE Error")
    axes[-1, col].xaxis.tick_bottom()
    axes[-1, col].set_xticks([0, 100])

plt.tight_layout()
plt.savefig("ope_std_scatter.pdf")  # Save as high-resolution image
plt.show()

# Calculate and print average RMSE for each model
average_rmse = {model: np.mean(rmses) for model, rmses in model_rmse.items()}
print("Average RMSE for each model:", average_rmse)

In [ ]:
ope_methods = ["is", "wis", "pdis", "wpdis", "direct"]
ope_labels = ["IS", "WIS", "PDIS", "WPDIS", "DM"]
models = ["edac", "awac", "bc", "td3_bc", "iql", "dqn", "cql-dqn"]
model_labels = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]

# Initialize a dictionary to store the standard deviations for each OPE method
ope_stds_agg = {method: [] for method in ope_methods}
outliers_agg = {method: [] for method in ope_methods}

# Process data for each model and each OPE method
for model in models:
    df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
    for method in ope_methods:
        ope_stds = df[f"std_{method}_estimate"]
        ope_stds_agg[method].extend(ope_stds)
        outliers_agg[method].extend(ope_stds[ope_stds > 100])

# Determine the common horizontal range with a cutoff at 100
min_std, max_std = 0, 100

# Create a figure with subplots
fig, axes = plt.subplots(ncols=1, nrows=len(ope_methods), figsize=(6, 8))

# Plot histograms for each OPE method
for idx, method in enumerate(ope_methods):
    axes[idx].hist(
        ope_stds_agg[method], bins=30, alpha=0.7, range=(min_std, max_std), log=True
    )
    axes[idx].set_title(f"{ope_labels[idx]} Estimate")
    axes[idx].set_xlabel("Standard Deviation")
    axes[idx].set_ylabel("Frequency")

plt.tight_layout()
plt.savefig("ope_std_histograms.pdf")  # Save as high-resolution image
plt.show()

# Print all outliers along with the associated OPE method
print("Outliers exceeding the range (0, 100):")
for method in ope_methods:
    for outlier in outliers_agg[method]:
        print(f"OPE Method: {method.upper()}, Outlier Value: {outlier}")

In [ ]:
ope_methods = ["is", "wis", "pdis", "wpdis", "direct"]
models = ["edac", "awac", "bc", "td3_bc", "iql", "dqn", "cql-dqn"]
model_labels = ["EDAC", "AWAC", "BC", "TD3+BC", "IQL", "DQN", "CQL"]

# Create a figure with subplots
fig, axes = plt.subplots(
    ncols=len(models), nrows=len(ope_methods), figsize=(6.75, 5.5)
)  # Adjust size as needed

# Initialize a dictionary to store RMSE values for each model
model_rmse = {model: [] for model in models}

# Process data for each model and each OPE method
for col, model in enumerate(models):
    for row, method in enumerate(ope_methods):
        df = pd.read_csv(os.path.join(results_directory, model + "_results.csv"))
        online_means = df["mean_return"]
        ope_means = df[f"mean_{method}_estimate"]

        # Clip the values to be between -100 and 100
        #online_means = np.clip(online_means, -100, 100)
        #ope_means = np.clip(ope_means, -100, 100)

        # Calculate RMSE and store it
        corr = np.corrcoef(online_means, ope_means)[0,1]
        print(model, method, corr)